# Dataset Balancing & Model Training (Phase 2)

## SECTION 1: Setup & Minimal Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import joblib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

print('✓ Minimal imports loaded')

✓ Minimal imports loaded


## SECTION 2: Load & Explore Data

In [3]:
# Try to locate a prepared features CSV automatically
possible_paths = [
    '../DATA_CLEANED/processed/engineered_features_scaled.csv',
    '../DATA_CLEANED/processed/engineered_features.csv',
    '../DATA_CLEANED/FIRE_CUT/fire_filtered.csv'
]

data_path = None
for p in possible_paths:
    if os.path.exists(p):
        data_path = p
        break

if data_path is None:
    print('⚠ No dataset found. Please set `data_path` to your features CSV.')
    df = None
else:
    print(f'✓ Loading dataset from: {data_path}')
    df = pd.read_csv(data_path)
    print('Shape:', df.shape)
    print('Columns:', list(df.columns)[:10], '...')
    display(df.head())

# Infer target column
TARGET_CANDIDATES = ['fire', 'label', 'target']
target_col = None
if df is not None:
    for c in TARGET_CANDIDATES:
        if c in df.columns:
            target_col = c
            break
    if target_col is None:
        print('⚠ Could not infer a target column from candidates:', TARGET_CANDIDATES)

# Separate features and target
if df is not None and target_col is not None:
    X = df.drop(columns=[target_col])
    y = df[target_col]
    print('Features shape:', X.shape, '| Target shape:', y.shape)
else:
    X, y = None, None

✓ Loading dataset from: ../DATA_CLEANED/processed/engineered_features_scaled.csv
Shape: (71050, 23)
Columns: ['fire', 'latitude', 'longitude', 'prec_p10', 'tmax_mean', 'tmax_std', 'tmin_mean', 'tmax_max', 't_range', 'gridcode'] ...


,fire,latitude,longitude,prec_p10,tmax_mean,tmax_std,tmin_mean,tmax_max,t_range,gridcode,...,ref_bulk,moisture_retention,texture_soter_M,elevation,terrain_roughness_index,bsat,ruggedness,teb,texture_soter_C,tcarbon_eq
0,0,36.395833,6.962500,1.180415,-0.807072,0.083139,-0.818977,-0.989303,-0.72555,-1.399386,...,1.393705,0.761360,0.760525,1.474982,-1.410052,0.738670,1.292073,0.896769,-0.412065,0.622275
1,0,37.312500,9.654167,0.747104,-0.807072,-1.262777,-0.818977,-1.377678,-0.72555,-1.271628,...,1.706704,0.256036,0.760525,-0.816444,0.821806,0.512056,-0.462006,0.507068,-0.412065,2.851375
2,0,36.095833,4.854167,1.491641,-0.807072,0.723202,-0.818977,-0.455287,-0.72555,-0.249563,...,1.706704,0.334186,0.760525,1.652124,0.821806,0.738670,-0.462006,1.119456,-0.412065,1.956702
3,0,36.645833,4.229167,1.184564,-0.807072,-0.597575,-0.818977,-1.183490,-0.72555,-1.399386,...,-0.379963,0.327643,0.760525,1.314982,-1.439894,0.058826,3.383477,-0.717708,-0.412065,-0.863791
4,0,36.637500,7.662500,0.928930,-0.807072,-0.670989,-0.818977,-0.843662,-0.72555,-1.399386,...,0.350371,-0.041119,0.760525,-0.816444,0.821806,0.766997,-0.462006,0.340053,-0.412065,0.667767


Features shape: (71050, 22) | Target shape: (71050,)


## SECTION 3: Stratified Train-Test Split

In [4]:
if X is None or y is None:
    print('⚠ Split skipped: define `X` and `y` in Section 2.')
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print('Train:', X_train.shape, '| Test:', X_test.shape)

    # Class ratio for info
    class_counts = y_train.value_counts()
    majority = class_counts.max()
    minority = class_counts.min()
    imbalance_ratio = majority / minority if minority > 0 else np.inf
    print('Imbalance ratio (train):', round(imbalance_ratio, 2))

Train: (56840, 22) | Test: (14210, 22)
Imbalance ratio (train): 4.0


## SECTION 7: EVALUATE & SELECT BEST MODEL

## SECTION 2: LOAD & EXPLORE DATA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BalancedRandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    precision_recall_curve, auc, f1_score, precision_score, recall_score,
    balanced_accuracy_score, accuracy_score
)
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print('✓ Libraries imported successfully')

## SECTION 1: SETUP & LIBRARY IMPORTS

# PHASE 2: DATASET BALANCING & MODEL TRAINING

## Fire Detection Model: Comprehensive ML Pipeline

This notebook implements dataset balancing and model training for fire detection in Algeria-Tunisia.

**Key Objectives:**
1. Load engineered features from Phase 1
2. Analyze and visualize class imbalance
3. Implement multiple balancing strategies (SMOTE, SMOTETomek, Class Weights)
4. Train and evaluate 5 diverse models
5. Perform cross-validation and select best model
6. Save results and produce final summary

## SECTION 4: Apply Balancing Strategies

In [4]:
# Ensure required variables exist
required = ['X_train', 'y_train']
if not all(name in globals() and globals()[name] is not None for name in required):
    print('⚠ Balancing skipped: run Sections 2–3 to define X_train/y_train.')
else:
    # Import here to keep top imports minimal
    from sklearn.utils.class_weight import compute_class_weight

    print('=' * 70)
    print('APPLYING BALANCING STRATEGIES')
    print('=' * 70)

    # Strategy 1: SMOTE
    smote = SMOTE(random_state=42, k_neighbors=5)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print(f'1. SMOTE: {len(y_train):,} → {len(y_train_smote):,} samples')

    # Strategy 2: SMOTETomek
    smo_tomek = SMOTETomek(random_state=42)
    X_train_smo_tomek, y_train_smo_tomek = smo_tomek.fit_resample(X_train, y_train)
    print(f'2. SMOTETomek: {len(y_train):,} → {len(y_train_smo_tomek):,} samples')

    # Strategy 3: Class Weights
    classes = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    class_weight_dict = {int(cls): float(w) for cls, w in zip(classes, class_weights)}
    print(f"3. Class Weights: Non-Fire={class_weight_dict.get(0):.2f}, Fire={class_weight_dict.get(1):.2f}")

APPLYING BALANCING STRATEGIES
1. SMOTE: 56,840 → 90,944 samples
2. SMOTETomek: 56,840 → 88,660 samples
3. Class Weights: Non-Fire=0.62, Fire=2.50


## SECTION 5: Train Models

In [ ]:
# Check that balancing outputs exist
needed = ['X_train_smote', 'y_train_smote', 'X_train_smo_tomek', 'y_train_smo_tomek', 'class_weight_dict']
if not all(name in globals() for name in needed):
    print('⚠ Training skipped: run Section 4 to generate balanced datasets and class weights.')
else:
    print('\n' + '=' * 70)
    print('TRAINING 5 MODELS × 3 STRATEGIES = 15 VARIANTS')
    print('=' * 70)

    # Define models
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1),
        'Balanced RF': BalancedRandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1),
        'SVM': SVC(kernel='rbf', probability=True, random_state=42)
    }

    # Strategy datasets
    strategies = {
        'SMOTE': (X_train_smote, y_train_smote),
        'SMOTETomek': (X_train_smo_tomek, y_train_smo_tomek),
        'Class Weights': (X_train, y_train, class_weight_dict)
    }

    trained_models = {}
    predictions_on_test = {}

    for strategy_name, data in strategies.items():
        print(f'\n{strategy_name}:')
        if strategy_name == 'Class Weights':
            X_s, y_s, weights = data
        else:
            X_s, y_s = data
            weights = None

        for model_name, base_model in models.items():
            print(f'  • {model_name}...', end=' ', flush=True)
            m = base_model.__class__(**base_model.get_params())
            if strategy_name == 'Class Weights' and hasattr(m, 'class_weight'):
                params = m.get_params()
                params['class_weight'] = weights
                m = m.__class__(**params)

            m.fit(X_s, y_s)
            y_pred = m.predict(X_test)
            y_pred_proba = m.predict_proba(X_test)[:, 1]

            key = f'{model_name}_{strategy_name}'
            trained_models[key] = m
            predictions_on_test[key] = {
                'y_pred': y_pred,
                'y_pred_proba': y_pred_proba,
                'model': model_name,
                'strategy': strategy_name
            }
            print('✓')

    print(f'\n✓ All models trained: {len(trained_models)} variants')


TRAINING 5 MODELS × 3 STRATEGIES = 15 VARIANTS

SMOTE:
  • Logistic Regression... ✓
  • Random Forest... ✓
  • Balanced RF... ✓
  • SVM... 

## SECTION 6: Evaluate & Select Best Model

In [ ]:
# Ensure predictions exist
if 'predictions_on_test' not in globals() or len(predictions_on_test) == 0:
    print('⚠ Evaluation skipped: run Section 5 to train models and generate predictions.')
else:
    print('\n' + '=' * 70)
    print('EVALUATING ALL MODELS')
    print('=' * 70)

    rows = []
    for key, preds in predictions_on_test.items():
        y_pred = preds['y_pred']
        y_pred_proba = preds['y_pred_proba']
        rows.append({
            'Model': preds['model'],
            'Strategy': preds['strategy'],
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred, zero_division=0),
            'Recall': recall_score(y_test, y_pred, zero_division=0),
            'F1': f1_score(y_test, y_pred, zero_division=0),
            'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
        })

    results_df = pd.DataFrame(rows)
    print('\nTop 5 Models by ROC-AUC:')
    print(results_df.nlargest(5, 'ROC-AUC')[['Model', 'Strategy', 'Accuracy', 'Recall', 'ROC-AUC']])

    best_idx = results_df['ROC-AUC'].idxmax()
    best_model_info = results_df.loc[best_idx]
    best_model_key = f"{best_model_info['Model']}_{best_model_info['Strategy']}"
    best_model_trained = trained_models[best_model_key]
    best_preds = predictions_on_test[best_model_key]

    print(f"\n🏆 BEST MODEL: {best_model_info['Model']} + {best_model_info['Strategy']}")
    print(f"   ROC-AUC: {best_model_info['ROC-AUC']:.4f}")
    print(f"   Recall: {best_model_info['Recall']:.4f}")

## SECTION 5A: Single-Model Timing (KNN, Decision Tree, Random Forest)

In [8]:
import time
from typing import Tuple

# Utility: choose training set per balancing mode
BALANCING_MODE = 'none'  # options: 'none', 'smote', 'smote_tomek', 'class_weights'

def get_train_data(mode: str) -> Tuple[pd.DataFrame, pd.Series, dict | None]:
    if mode == 'smote':
        if 'X_train_smote' in globals():
            return X_train_smote, y_train_smote, None
        else:
            print('⚠ SMOTE data not found. Falling back to none.')
            return X_train, y_train, None
    if mode == 'smote_tomek':
        if 'X_train_smo_tomek' in globals():
            return X_train_smo_tomek, y_train_smo_tomek, None
        else:
            print('⚠ SMOTETomek data not found. Falling back to none.')
            return X_train, y_train, None
    if mode == 'class_weights':
        if 'class_weight_dict' in globals():
            return X_train, y_train, class_weight_dict
        else:
            print('⚠ class_weight_dict not found. Falling back to none.')
            return X_train, y_train, None
    return X_train, y_train, None

# Utility: time fit and predict
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

def time_model_fit_predict(model, X_tr, y_tr, X_te, y_te):
    t0 = time.perf_counter()
    model.fit(X_tr, y_tr)
    t_fit = time.perf_counter() - t0

    t1 = time.perf_counter()
    y_pred = model.predict(X_te)
    t_pred = time.perf_counter() - t1

    # proba or decision
    if hasattr(model, 'predict_proba'):
        y_proba = model.predict_proba(X_te)[:, 1]
    elif hasattr(model, 'decision_function'):
        y_score = model.decision_function(X_te)
        # Map to [0,1] via a simple min-max if needed
        y_proba = (y_score - y_score.min()) / (y_score.max() - y_score.min() + 1e-9)
    else:
        y_proba = None

    metrics = {
        'accuracy': accuracy_score(y_te, y_pred),
        'precision': precision_score(y_te, y_pred, zero_division=0),
        'recall': recall_score(y_te, y_pred, zero_division=0),
        'f1': f1_score(y_te, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_te, y_proba) if y_proba is not None else None,
    }
    return metrics, t_fit, t_pred

### Run: KNN (timed)

In [13]:
# Imports local to this cell to keep top-level minimal
from sklearn.neighbors import KNeighborsClassifier

X_tr, y_tr, weights = get_train_data(BALANCING_MODE)
print(f'KNN using training set: {X_tr.shape}, BALANCING_MODE={BALANCING_MODE}')

knn = KNeighborsClassifier(n_neighbors=15, weights='distance', n_jobs=-1)
metrics, t_fit, t_pred = time_model_fit_predict(knn, X_tr, y_tr, X_test, y_test)

print(f"Fit time:  {t_fit:.2f}s (KNN fit is typically trivial)")
print(f"Pred time: {t_pred:.2f}s (KNN prediction scales with train size)")
print('Metrics:', metrics)

KNN using training set: (90944, 22), BALANCING_MODE=smote
Fit time:  0.02s (KNN fit is typically trivial)
Pred time: 2.53s (KNN prediction scales with train size)
Metrics: {'accuracy': 0.8898662913441239, 'precision': 0.6801128349788435, 'recall': 0.8483462350457425, 'f1': 0.7549710349146704, 'roc_auc': np.float64(0.9091636102377081)}


### Run: Decision Tree (timed)

In [14]:
from sklearn.tree import DecisionTreeClassifier

X_tr, y_tr, weights = get_train_data(BALANCING_MODE)
print(f'DecisionTree using training set: {X_tr.shape}, BALANCING_MODE={BALANCING_MODE}')

params = {'random_state': 42, 'max_depth': None}
if BALANCING_MODE == 'class_weights' and weights is not None:
    params['class_weight'] = weights

dt = DecisionTreeClassifier(**params)
metrics, t_fit, t_pred = time_model_fit_predict(dt, X_tr, y_tr, X_test, y_test)

print(f"Fit time:  {t_fit:.2f}s")
print(f"Pred time: {t_pred:.2f}s")
print('Metrics:', metrics)

DecisionTree using training set: (90944, 22), BALANCING_MODE=smote
Fit time:  0.96s
Pred time: 0.01s
Metrics: {'accuracy': 0.926460239268121, 'precision': 0.8016112789526687, 'recall': 0.840253342716397, 'f1': 0.82047758117162, 'roc_auc': np.float64(0.8941326530612246)}


### Run: Random Forest (timed)

In [15]:
# Random Forest
X_tr, y_tr, weights = get_train_data(BALANCING_MODE)
print(f'RandomForest using training set: {X_tr.shape}, BALANCING_MODE={BALANCING_MODE}')

params = {
    'n_estimators': 200,
    'random_state': 42,
    'n_jobs': -1,
}
# class weights supported: 'balanced' or dict
if BALANCING_MODE == 'class_weights' and weights is not None:
    params['class_weight'] = weights

rf = RandomForestClassifier(**params)
metrics, t_fit, t_pred = time_model_fit_predict(rf, X_tr, y_tr, X_test, y_test)

print(f"Fit time:  {t_fit:.2f}s")
print(f"Pred time: {t_pred:.2f}s")
print('Metrics:', metrics)

RandomForest using training set: (90944, 22), BALANCING_MODE=smote
Fit time:  9.53s
Pred time: 0.16s
Metrics: {'accuracy': 0.9482054890921886, 'precision': 0.9000759878419453, 'recall': 0.8335679099225898, 'f1': 0.865546218487395, 'roc_auc': np.float64(0.930210720265684)}


## SECTION 5B: Toggle Balancing Mode

In [12]:
# Set the balancing mode here and ensure needed data exists
BALANCING_MODE = 'smote'  # options: 'none' | 'smote' | 'smote_tomek' | 'class_weights'
print('BALANCING_MODE =', BALANCING_MODE)

# Ensure balanced datasets when needed
if BALANCING_MODE in ('smote', 'smote_tomek', 'class_weights'):
    from imblearn.over_sampling import SMOTE
    from imblearn.combine import SMOTETomek
    from sklearn.utils.class_weight import compute_class_weight
    import numpy as np

    if 'X_train_smote' not in globals() and BALANCING_MODE in ('smote', 'smote_tomek'):
        smote = SMOTE(random_state=42, k_neighbors=5)
        X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
        print('✓ SMOTE data prepared:', X_train_smote.shape)

    if 'X_train_smo_tomek' not in globals() and BALANCING_MODE == 'smote_tomek':
        smo_tomek = SMOTETomek(random_state=42)
        X_train_smo_tomek, y_train_smo_tomek = smo_tomek.fit_resample(X_train, y_train)
        print('✓ SMOTETomek data prepared:', X_train_smo_tomek.shape)

    if 'class_weight_dict' not in globals() and BALANCING_MODE == 'class_weights':
        classes = np.unique(y_train)
        class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        class_weight_dict = {int(cls): float(w) for cls, w in zip(classes, class_weights)}
        print('✓ class_weight_dict prepared:', class_weight_dict)

BALANCING_MODE = smote
✓ SMOTE data prepared: (90944, 22)


# Summary: Data Balancing + Supervised Training

- Approach: Tomek Links + ENN applied only on training folds; RandomForest with `class_weight='balanced_subsample'`; evaluated via `StratifiedKFold`.

- CV results (RF best): ROC-AUC ≈ 0.917; PR-AUC ≈ 0.889; F1 ≈ 0.795 at default threshold.

- Threshold tuning (maximize F1 via PR curve):
  - Recommended threshold ≈ 0.77–0.78 (mean 0.7715, std 0.0283).
  - At tuned threshold: Precision ≈ 0.929, Recall ≈ 0.770, F1 ≈ 0.842; ROC-AUC ≈ 0.9166; PR-AUC ≈ 0.8885.

- Operational guidance:
  - Prefer fewer false alarms → raise threshold to ~0.80–0.81.
  - Prefer fewer misses → lower threshold to ~0.74–0.75.

- Next steps: Holdout validation, export model + threshold, inference utility, feature importance.

In [ ]:
import json
from pathlib import Path

results_path = Path('f:/DATA/results/rf_tomek_enn_threshold_tuning.json')
if results_path.exists():
    data = json.loads(results_path.read_text())
    agg = data.get('aggregated', {})
    threshold = agg.get('threshold_mean', 0.77)
    print(f"Recommended RF threshold: {threshold:.4f}")
    print("Aggregated metrics @ tuned threshold:")
    for k in ('roc_auc_mean','pr_auc_mean','precision_mean','recall_mean','f1_mean'):
        print(f"  {k}: {agg.get(k):.4f}")
else:
    print('Threshold tuning results JSON not found at:', results_path)